In [2]:

import numpy as np
import pandas as pd
from collections import Counter
import math, os, joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier as SKDecisionTree

In [3]:
def load_adult():
    try:
        from ucimlrepo import fetch_ucirepo
        adult = fetch_ucirepo(id=2)
        X = adult.data.features
        y = adult.data.targets
        if isinstance(y, pd.DataFrame):
            y = y.iloc[:,0]
        return X, y
    except Exception as e:
        print("ucimlrepo unavailable or fetch failed:", e)
        try:
            from sklearn.datasets import fetch_openml
            data = fetch_openml(name='adult', version=2, as_frame=True)
            X = data.data
            y = data.target
            return X, y
        except Exception as e2:
            print("OpenML fallback failed:", e2)
            raise RuntimeError("Could not fetch Adult dataset. Please run in an environment with internet or provide the data file.")

In [4]:
class TreeNode:
    def __init__(self, depth=0):
        self.depth = depth
        self.is_leaf = False
        self.prediction = None
        self.feature_index = None
        self.threshold = None
        self.is_categorical = False
        self.left = None
        self.right = None
        self.class_counts = None
        self.samples = 0

In [5]:
class DecisionTreeFromScratch:
    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2, min_impurity_decrease=1e-7):
        assert criterion in ('gini', 'entropy')
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.root = None

    def _impurity(self, y):
        counts = np.bincount(y, minlength=1)
        probs = counts[counts>0] / y.size
        if self.criterion == 'gini':
            return 1.0 - np.sum(probs**2)
        else:
            return -np.sum(probs * np.log2(probs))

    def _weighted_impurity(self, y_left, y_right):
        n = y_left.size + y_right.size
        return (y_left.size/n)*self._impurity(y_left) + (y_right.size/n)*self._impurity(y_right)

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples < self.min_samples_split:
            return None
        parent_impurity = self._impurity(y)
        best_gain = 0.0
        best = None

        for feat in range(n_features):
            values = X[:, feat]
            unique_vals = np.unique(values)
            if unique_vals.size == 1:
                continue

            is_categorical = (unique_vals.size <= 15 and np.all(unique_vals == np.floor(unique_vals)))
            if is_categorical:
                for val in unique_vals:
                    left_idx = np.where(values == val)[0]
                    right_idx = np.where(values != val)[0]
                    if left_idx.size == 0 or right_idx.size == 0:
                        continue
                    impurity = self._weighted_impurity(y[left_idx], y[right_idx])
                    gain = parent_impurity - impurity
                    if gain > best_gain + 1e-12:
                        best_gain = gain
                        best = (feat, val, True, left_idx, right_idx)
            else:
                sorted_idx = np.argsort(values)
                sorted_vals = values[sorted_idx]
                sorted_y = y[sorted_idx]
                for i in range(1, n_samples):
                    if sorted_vals[i] == sorted_vals[i-1]:
                        continue
                    thresh = (sorted_vals[i] + sorted_vals[i-1]) / 2.0
                    left_idx = np.where(values <= thresh)[0]
                    right_idx = np.where(values > thresh)[0]
                    if left_idx.size == 0 or right_idx.size == 0:
                        continue
                    impurity = self._weighted_impurity(y[left_idx], y[right_idx])
                    gain = parent_impurity - impurity
                    if gain > best_gain + 1e-12:
                        best_gain = gain
                        best = (feat, thresh, False, left_idx, right_idx)

        if best is None or best_gain < self.min_impurity_decrease:
            return None
        feat, thresh, is_cat, left_idx, right_idx = best
        return feat, thresh, is_cat, left_idx, right_idx

    def _build(self, X, y, depth=0):
        node = TreeNode(depth=depth)
        node.samples = y.size
        counts = np.bincount(y, minlength=2)
        node.class_counts = counts
        node.prediction = np.argmax(counts)
        # stopping conditions
        if np.unique(y).size == 1:
            node.is_leaf = True
            return node
        if self.max_depth is not None and depth >= self.max_depth:
            node.is_leaf = True
            return node
        if y.size < self.min_samples_split:
            node.is_leaf = True
            return node

        split = self._best_split(X, y)
        if split is None:
            node.is_leaf = True
            return node

        feat, thresh, is_cat, left_idx, right_idx = split
        node.feature_index = feat
        node.threshold = thresh
        node.is_categorical = is_cat

        node.left = self._build(X[left_idx], y[left_idx], depth+1)
        node.right = self._build(X[right_idx], y[right_idx], depth+1)
        return node

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y))
        self.n_features_in_ = X.shape[1]
        self.root = self._build(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        while not node.is_leaf:
            if node.is_categorical:
                if x[node.feature_index] == node.threshold:
                    node = node.left
                else:
                    node = node.right
            else:
                if x[node.feature_index] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
        return node.prediction

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

    def post_prune(self, X_val, y_val):
        """
        Reduced error pruning:
        - consider every internal node; if replacing it by leaf (majority class) does not reduce val accuracy, keep the replacement.
        - repeat until no improvement
        """
        improved = True
        while improved:
            improved = False
            current_acc = accuracy_score(y_val, self.predict(X_val))
            # get list of internal nodes
            stack = [(self.root, None, None)]
            candidates = []
            while stack:
                node, parent, is_left = stack.pop()
                if node is None or node.is_leaf:
                    continue
                candidates.append((node, parent, is_left))
                stack.append((node.left, node, True))
                stack.append((node.right, node, False))

            best_candidate = None
            for node, parent, is_left in candidates:
                # simulate pruning
                # backup
                if parent is None:
                    saved_root = self.root
                else:
                    saved_child = parent.left if is_left else parent.right
                # create leaf
                new_leaf = TreeNode(depth=node.depth)
                new_leaf.is_leaf = True
                new_leaf.prediction = int(np.argmax(node.class_counts))
                # attach
                if parent is None:
                    self.root = new_leaf
                else:
                    if is_left:
                        parent.left = new_leaf
                    else:
                        parent.right = new_leaf
                # evaluate
                acc = accuracy_score(y_val, self.predict(X_val))
                # restore
                if parent is None:
                    self.root = saved_root
                else:
                    if is_left:
                        parent.left = saved_child
                    else:
                        parent.right = saved_child
                if acc >= current_acc - 1e-12:
                    current_acc = acc
                    best_candidate = (node, parent, is_left)

            if best_candidate is not None:
                node, parent, is_left = best_candidate
                leaf = TreeNode(depth=node.depth)
                leaf.is_leaf = True
                leaf.prediction = int(np.argmax(node.class_counts))
                if parent is None:
                    self.root = leaf
                else:
                    if is_left:
                        parent.left = leaf
                    else:
                        parent.right = leaf
                improved = True

In [6]:
def evaluate_model(model, X, y):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(y, preds, average='binary', zero_division=0)
    cm = confusion_matrix(y, preds)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "confusion_matrix": cm}

def main():
    X_raw, y_raw = load_adult()
    print("Loaded data shape:", X_raw.shape, getattr(y_raw, "shape", None))

    # quick cleaning: replace '?' with NaN, then drop rows with missing values
    X = X_raw.copy().reset_index(drop=True)
    y = y_raw.copy().reset_index(drop=True)
    X = X.replace('?', np.nan)
    missing_before = X.isna().sum().sum()
    print("Missing values in features (total):", missing_before)
    X = X.dropna().reset_index(drop=True)
    y = y.loc[X.index].reset_index(drop=True)
    print("After dropping missing rows:", X.shape)

    # label encode categorical features
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_cols = [c for c in X.columns if c not in categorical_cols]
    print("Categorical cols:", len(categorical_cols), "Numeric cols:", len(numeric_cols))
    encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        encoders[col] = le
    target_le = LabelEncoder()
    y = target_le.fit_transform(y.astype(str))

    # Train / val / test split
    # NOTE: assignment text is ambiguous; we choose 60/20/20. To make it 80/10/10,
    # change the split parameters below accordingly.
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)
    print("Shapes -> train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)

    # convert to numpy arrays for our implementation
    X_train_np = X_train.values
    X_val_np = X_val.values
    X_test_np = X_test.values
    feature_names = X.columns.tolist()

    # Experiments
    depths = [2,4,6,None]
    criteria = ['gini', 'entropy']
    results = []

    for crit in criteria:
        for depth in depths:
            # full (no pre-pruning)
            dt_full = DecisionTreeFromScratch(criterion=crit, max_depth=None, min_samples_split=2)
            dt_full.fit(X_train_np, y_train)
            res_full = evaluate_model(dt_full, X_test_np, y_test)
            res_full['name'] = f"Scratch-{crit}-full"

            # pre-pruned (max depth = depth, min_samples_split=5)
            dt_pre = DecisionTreeFromScratch(criterion=crit, max_depth=depth, min_samples_split=5)
            dt_pre.fit(X_train_np, y_train)
            res_pre = evaluate_model(dt_pre, X_test_np, y_test)
            res_pre['name'] = f"Scratch-{crit}-pre_d{depth}"

            # post-pruned: grow full then prune using validation set
            dt_post = DecisionTreeFromScratch(criterion=crit, max_depth=None, min_samples_split=2)
            dt_post.fit(X_train_np, y_train)
            dt_post.post_prune(X_val_np, y_val)
            res_post = evaluate_model(dt_post, X_test_np, y_test)
            res_post['name'] = f"Scratch-{crit}-postpruned"

            # sklearn baseline (matching criterion and max_depth)
            skl = SKDecisionTree(criterion='gini' if crit=='gini' else 'entropy', max_depth=depth, min_samples_split=5, random_state=42)
            skl.fit(X_train, y_train)
            skl_preds = skl.predict(X_test)
            acc = accuracy_score(y_test, skl_preds)
            prec, rec, f1, _ = precision_recall_fscore_support(y_test, skl_preds, average='binary', zero_division=0)
            cm = confusion_matrix(y_test, skl_preds)
            res_skl = {'name': f'Sklearn-{crit}-maxdepth_{depth}', 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'confusion_matrix': cm}

            results.extend([res_full, res_pre, res_post, res_skl])
            print("-- done:", crit, "depth", depth)


In [7]:
def main():
    X_raw, y_raw = load_adult()
    print("Loaded data shape:", X_raw.shape, getattr(y_raw, "shape", None))

    # quick cleaning: replace '?' with NaN, then drop rows with missing values
    X = X_raw.copy().reset_index(drop=True)
    y = y_raw.copy().reset_index(drop=True)
    X = X.replace('?', np.nan)
    missing_before = X.isna().sum().sum()
    print("Missing values in features (total):", missing_before)
    X = X.dropna().reset_index(drop=True)
    y = y.loc[X.index].reset_index(drop=True)
    print("After dropping missing rows:", X.shape)

    # label encode categorical features
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_cols = [c for c in X.columns if c not in categorical_cols]
    print("Categorical cols:", len(categorical_cols), "Numeric cols:", len(numeric_cols))
    encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        encoders[col] = le
    target_le = LabelEncoder()
    y = target_le.fit_transform(y.astype(str))

    # Train / val / test split
    # NOTE: assignment text is ambiguous; we choose 60/20/20. To make it 80/10/10,
    # change the split parameters below accordingly.
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)
    print("Shapes -> train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)

    # convert to numpy arrays for our implementation
    X_train_np = X_train.values
    X_val_np = X_val.values
    X_test_np = X_test.values
    feature_names = X.columns.tolist()

    # Experiments
    depths = [2,4,6,None]
    criteria = ['gini', 'entropy']
    results = []

    for crit in criteria:
        for depth in depths:
            # full (no pre-pruning)
            dt_full = DecisionTreeFromScratch(criterion=crit, max_depth=None, min_samples_split=2)
            dt_full.fit(X_train_np, y_train)
            res_full = evaluate_model(dt_full, X_test_np, y_test)
            res_full['name'] = f"Scratch-{crit}-full"

            # pre-pruned (max depth = depth, min_samples_split=5)
            dt_pre = DecisionTreeFromScratch(criterion=crit, max_depth=depth, min_samples_split=5)
            dt_pre.fit(X_train_np, y_train)
            res_pre = evaluate_model(dt_pre, X_test_np, y_test)
            res_pre['name'] = f"Scratch-{crit}-pre_d{depth}"

            # post-pruned: grow full then prune using validation set
            dt_post = DecisionTreeFromScratch(criterion=crit, max_depth=None, min_samples_split=2)
            dt_post.fit(X_train_np, y_train)
            dt_post.post_prune(X_val_np, y_val)
            res_post = evaluate_model(dt_post, X_test_np, y_test)
            res_post['name'] = f"Scratch-{crit}-postpruned"

            # sklearn baseline (matching criterion and max_depth)
            skl = SKDecisionTree(criterion='gini' if crit=='gini' else 'entropy', max_depth=depth, min_samples_split=5, random_state=42)
            skl.fit(X_train, y_train)
            skl_preds = skl.predict(X_test)
            acc = accuracy_score(y_test, skl_preds)
            prec, rec, f1, _ = precision_recall_fscore_support(y_test, skl_preds, average='binary', zero_division=0)
            cm = confusion_matrix(y_test, skl_preds)
            res_skl = {'name': f'Sklearn-{crit}-maxdepth_{depth}', 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'confusion_matrix': cm}

            results.extend([res_full, res_pre, res_post, res_skl])
            print("-- done:", crit, "depth", depth)

    summary = []
    for r in results:
        summary.append({'Model': r['name'], 'Accuracy': round(r['accuracy'],4), 'Precision': round(r['precision'],4), 'Recall': round(r['recall'],4), 'F1': round(r['f1'],4)})
    summary_df = pd.DataFrame(summary)
    print("\nSUMMARY:")
    print(summary_df.sort_values('Accuracy', ascending=False).reset_index(drop=True))

    # Root feature from last dt_full (gini or entropy depending loop order)
    try:
        root_info = (feature_names[dt_full.root.feature_index], dt_full.root.threshold, dt_full.root.is_categorical)
        print("\nRoot feature used by last full scratch tree:", root_info)
    except Exception:
        print("Could not extract root info.")

    # show sklearn feature importances
    skl_full = SKDecisionTree(criterion='gini', random_state=42)
    skl_full.fit(X_train, y_train)
    feat_imp = pd.Series(skl_full.feature_importances_, index=feature_names).sort_values(ascending=False)
    print("\nTop sklearn feature importances:\n", feat_imp.head(10))

    # Save outputs
    out_dir = "./decision_tree_outputs"
    os.makedirs(out_dir, exist_ok=True)
    joblib.dump(dt_full, os.path.join(out_dir, 'dt_full_scratch.joblib'))
    joblib.dump(dt_post, os.path.join(out_dir, 'dt_post_scratch.joblib'))
    joblib.dump(skl_full, os.path.join(out_dir, 'skl_full.joblib'))
    joblib.dump(encoders, os.path.join(out_dir, 'encoders.joblib'))
    joblib.dump(target_le, os.path.join(out_dir, 'target_le.joblib'))
    summary_df.to_csv(os.path.join(out_dir, 'experiment_summary.csv'), index=False)
    print("\nSaved models & results to:", out_dir)

    # Print classification report for best model by accuracy
    best = max(results, key=lambda x: x['accuracy'])
    print("\nBest model on test:", best['name'], "Accuracy:", best['accuracy'])
    # print confusion matrix & classification report
    print("Confusion matrix:\n", best['confusion_matrix'])
    # get classification report
    # For the best model, pick the model instance accordingly (approx)
    if best['name'].startswith('Scratch') and 'post' in best['name']:
        report_preds = dt_post.predict(X_test_np)
    elif best['name'].startswith('Scratch') and 'pre' in best['name']:
        report_preds = dt_pre.predict(X_test_np)
    elif best['name'].startswith('Scratch') and 'full' in best['name']:
        report_preds = dt_full.predict(X_test_np)
    else:
        report_preds = skl_full.predict(X_test)
    print("\nClassification report (best model):\n", classification_report(y_test, report_preds, zero_division=0))

In [ ]:
if __name__ == "__main__":
    main()

ucimlrepo unavailable or fetch failed: No module named 'ucimlrepo'
Loaded data shape: (48842, 14) (48842,)
Missing values in features (total): 6465
After dropping missing rows: (45222, 14)
Categorical cols: 8 Numeric cols: 6
Shapes -> train: (27132, 14) val: (9045, 14) test: (9045, 14)
-- done: gini depth 2
-- done: gini depth 4
-- done: gini depth 6
-- done: gini depth None
